## Import Libraries

In [ ]:
import sys,os

RES_PATH = 'models' # Folder where all 3D models are kept. Same dir level as this file.

if not os.path.exists(RES_PATH):
    print(f"cannot find {RES_PATH} folder, please update RES_PATH")
    exit(1)
else:
    pass

import pyglet
pyglet.options['shadow_window'] = False

import pyrender
import numpy as np
import trimesh

import matplotlib
import matplotlib.pyplot as plt

from sklearn.neighbors import KDTree

%load_ext autoreload
%autoreload 2

## Trimesh Utility

In [ ]:
def load_mesh(filename):
    mesh_fp = os.path.join(RES_PATH, filename)
    
    assert os.path.exists(mesh_fp), 'cannot found:'+mesh_fp
    
    t_mesh = trimesh.load(mesh_fp)
    
    return t_mesh

## Mesh Simplification Utility

## Main Code

In [ ]:
def simplify_mesh(mesh):
    pass